# <h1><center><font size='7'>Leaves Diseases 32 Classes</font></center></h1>

***I used a pretrained model named VGG16 instead of creating the layers myself since there are 32 classes which is a lot relatively.
Since the usage of GPU limitations of Colab, I had to train the model first 5 epochs. Then, uploading the model again and continue to train in a different account. In total 25 epochs were done by loading the model 3 times (5,5,5,10).
After training the model, I achieved a model that distinguish diseases of leaves with nearly %78 accuracy.***

In [1]:
# since it takes too much time to upload the dataset to colab. First, the dataset was turned into a zip file and then uploaded
# to Google Drive. Finally, imported in this code.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# unzipped the file
!unzip "/content/gdrive/MyDrive/TRAIN_LEAVES.zip"

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1149).jpg  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1150).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1151).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1152).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1153).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1154).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1155).jpg  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1156).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1157).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1158).JPG  
  inflating: TRAIN_LEAVES/validation/Tomato_Late_Blight/TomatoLateBlight(1159).JPG  
  inflating: TRAIN

In [3]:
# shows that GPU is active
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
import os
import matplotlib.image as mpimg
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import matplotlib.pyplot as plt

In [5]:
path = r"/content/TRAIN_LEAVES/train"

# get the list of file names in the specified path
file_names = os.listdir(path)

# extract the class names from the file names
classes = [file_name.split('.')[0] for file_name in file_names]

# remove duplicates and sort the class names
classes = sorted(list(set(classes)))
class_number = len(classes)
print(classes)
print(class_number)

['Apple_Black_Rot', 'Apple_Cedar_Rust', 'Apple_Healthy', 'Apple_Scab', 'Cherry_Healthy', 'Cherry_Powdery_Mildew', 'Corn_Common_Rust', 'Corn_Healthy', 'Corn_Northern_Leaf_Blight', 'Grape_Black_Rot', 'Grape_Esca_(Black_Measles)', 'Grape_Healthy', 'Grape_Leaf_Blight_(Isariopsis_Leaf_Spot)', 'Peach_Bacterial_Spot', 'Peach_Healthy', 'Pepper_Bacterial_Spot', 'Pepper_Healthy', 'Potato_Early_Blight', 'Potato_Healthy', 'Potato_Late_Blight', 'Strawberry_Healthy', 'Strawberry_Leaf_Scorch', 'Tomato_Bacterial_Spot', 'Tomato_Early_Blight', 'Tomato_Healthy', 'Tomato_Late_Blight', 'Tomato_Leaf_Mold', 'Tomato_Mosaic_Virus', 'Tomato_Septoria_Leaf_Spot', 'Tomato_Target_Spot', 'Tomato_Two_Spotted_Spider_Mite', 'Tomato_Yellow_Leaf_Curl_Virus']
32


In [6]:
label_names_txt = "leaf_diseases_32_classes.txt"
# write class names and indices to the text file
with open(label_names_txt, "w") as file:
    for index, class_name in enumerate(classes):
        file.write(f"{index} {class_name}\n")

In [7]:
# train and validation data paths
train_dir = os.path.join(r"/content/TRAIN_LEAVES/train")
validation_dir = os.path.join(r'/content/TRAIN_LEAVES/validation')
test_dir = os.path.join(r"/content/TRAIN_LEAVES/test")

trains = []
validations = []

# train and validation data paths for each class are stored
for i in range(len(classes)):
    trains.append(os.path.join(train_dir, classes[i]))
    validations.append(os.path.join(validation_dir, classes[i]))

In [8]:
# add the data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [9]:
# flow training images in batches of 256 using train_datagen generator (%60 of the dataset)
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 256, class_mode = 'categorical', target_size = (224, 224))

# flow validation images in batches of 256 using test_datagen generator (%20 of the dataset)
validation_generator = test_datagen.flow_from_directory(validation_dir,  batch_size = 256, class_mode = 'categorical', target_size = (224, 224))

# flow test images in batches of 256 using test_datagen generator (%20 of the dataset)
test_generator = test_datagen.flow_from_directory(test_dir,  batch_size = 256, class_mode = 'categorical', target_size = (224, 224))

Found 67352 images belonging to 32 classes.
Found 22425 images belonging to 32 classes.
Found 22425 images belonging to 32 classes.


In [10]:
# retrieve the mapping of class labels to numerical indices
class_indices = train_generator.class_indices

# print the class indices
print(class_indices)

{'Apple_Black_Rot': 0, 'Apple_Cedar_Rust': 1, 'Apple_Healthy': 2, 'Apple_Scab': 3, 'Cherry_Healthy': 4, 'Cherry_Powdery_Mildew': 5, 'Corn_Common_Rust': 6, 'Corn_Healthy': 7, 'Corn_Northern_Leaf_Blight': 8, 'Grape_Black_Rot': 9, 'Grape_Esca_(Black_Measles)': 10, 'Grape_Healthy': 11, 'Grape_Leaf_Blight_(Isariopsis_Leaf_Spot)': 12, 'Peach_Bacterial_Spot': 13, 'Peach_Healthy': 14, 'Pepper_Bacterial_Spot': 15, 'Pepper_Healthy': 16, 'Potato_Early_Blight': 17, 'Potato_Healthy': 18, 'Potato_Late_Blight': 19, 'Strawberry_Healthy': 20, 'Strawberry_Leaf_Scorch': 21, 'Tomato_Bacterial_Spot': 22, 'Tomato_Early_Blight': 23, 'Tomato_Healthy': 24, 'Tomato_Late_Blight': 25, 'Tomato_Leaf_Mold': 26, 'Tomato_Mosaic_Virus': 27, 'Tomato_Septoria_Leaf_Spot': 28, 'Tomato_Target_Spot': 29, 'Tomato_Two_Spotted_Spider_Mite': 30, 'Tomato_Yellow_Leaf_Curl_Virus': 31}


**The VGG 16 model was uploaded only starting training with the first 5 epochs.**

In [ ]:
# load VGG16 model (In the first 5 epoch it was used)
# the model hasn't been trained yet.
base_model = VGG16(input_shape = (224, 224, 3),
# leave out the last fully connected layer
include_top = False,
weights = 'imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# set the 'trainable' property of the layer to False
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# add a final softmax layer
x = layers.Dense(class_number, activation='softmax')(x)

# model is stored here
model = tf.keras.models.Model(base_model.input, x)

# compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])

**Otherwise, the saved model was uploaded and continued to train with more epochs.**

In [ ]:
# the model was loaded and continued to train more epochs.
from tensorflow.keras.models import load_model

# Load the saved model (the model trained with 15 epochs was uploaded in here)
loaded_model = load_model("leaf_diseases_32_classes(2).h5")

**In the second loaded model, I forgot to run the code below, so it started to train from the beginning but I think it was not that much problem.**

In [16]:
# set the 'trainable' property of the layer to False
for layer in loaded_model.layers:
    layer.trainable = False

**The learning rate was increased in here. In the first loaded model, it was made 0.01. Then, it was made 0.1 to better learning.**

In [17]:
# flatten the output layer to 1 dimension
x = layers.Flatten()(loaded_model.output)

# add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# add a final softmax layer
x = layers.Dense(class_number, activation='softmax')(x)

# model is stored here
model = tf.keras.models.Model(loaded_model.input, x)

# compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['acc'])

In [18]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [19]:
# train the model 10 epoch (it shows the last loaded model training with 10 epochs)
vgghist = loaded_model.fit(train_generator, validation_data = validation_generator, epochs = 10)

Epoch 1/10
264/264 [==============================] - 1232s 5s/step - loss: 1.2828 - acc: 0.6063 - val_loss: 0.6473 - val_acc: 0.7996
Epoch 2/10
264/264 [==============================] - 1115s 4s/step - loss: 1.2749 - acc: 0.6080 - val_loss: 0.6805 - val_acc: 0.7847
Epoch 3/10
264/264 [==============================] - 1111s 4s/step - loss: 1.2685 - acc: 0.6087 - val_loss: 0.5795 - val_acc: 0.8161
Epoch 4/10
264/264 [==============================] - 1113s 4s/step - loss: 1.2512 - acc: 0.6133 - val_loss: 0.6257 - val_acc: 0.7978
Epoch 5/10
264/264 [==============================] - 1127s 4s/step - loss: 1.2328 - acc: 0.6170 - val_loss: 0.5727 - val_acc: 0.8177
Epoch 6/10
264/264 [==============================] - 1144s 4s/step - loss: 1.2250 - acc: 0.6182 - val_loss: 0.6120 - val_acc: 0.8128
Epoch 7/10
264/264 [==============================] - 1132s 4s/step - loss: 1.2199 - acc: 0.6185 - val_loss: 0.5892 - val_acc: 0.8164
Epoch 8/10
264/264 [==============================] - 1181s 4s

In [20]:
# save the model
loaded_model.save("leaf_diseases_32_classes(3).h5")

In [11]:
from tensorflow.keras.models import load_model

# Load the saved model to see if there is any problem
loaded_model_new = load_model("leaf_diseases_32_classes(3).h5")

In [21]:
# evaluate the test data to see the accuracy and loss of the model
loaded_model_new.evaluate(test_generator)

88/88 [==============================] - 104s 1s/step - loss: 0.6900 - acc: 0.7841


[0.689976155757904, 0.7841248512268066]

In [ ]:
# load and save the model to tflite
model = tf.keras.models.load_model('leaf_diseases_32_classes(3).h5', compile=False)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("leaf_diseases_32_classes_GPU.h5.tflite", "wb").write(tflite_model)